In [2]:
import pickle as pkl
import numpy as np
from glob import glob

In [3]:
train_datasets = ["cityscapes_train", "sim10k"]
test_datasets = ["cityscapes", "sim200k"]
ngc_id = 2838351

In [9]:
train_features = []
train_labels = []
for i, dset in enumerate(train_datasets):
    p = list(glob(f"/home/andliao/results/extract_features/*/{ngc_id}_{dset}.pkl"))
    assert len(p) == 1
    p = p[0]

    outs = pkl.load(open(p, "rb"))
    for out in outs:
        feat = out[-1]
        train_features.append(feat[0].mean(-1).mean(-1))     # average pooling
        train_labels.append(i)


train_features = np.array(train_features)
train_labels = np.array(train_labels)

In [10]:
test_features = []
test_labels = []
for i, dset in enumerate(test_datasets):
    p = list(glob(f"/home/andliao/results/extract_features/*/{ngc_id}_{dset}.pkl"))
    assert len(p) == 1
    p = p[0]

    outs = pkl.load(open(p, "rb"))
    for out in outs:
        feat = out[-1]
        test_features.append(feat[0].mean(-1).mean(-1))     # average pooling
        test_labels.append(i)

test_features = np.array(test_features)
test_labels = np.array(test_labels)

In [11]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(train_features, train_labels)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [14]:
acc = np.mean(clf.predict(test_features) == test_labels)

0.7824113475177304